In [12]:
import numpy as np
import gym
from numpy.random import choice
import random
from scipy.interpolate import interp1d

env = gym.make("FrozenLake-v0")

[2017-03-02 10:15:20,417] Making new env: FrozenLake-v0


In [13]:
def next_action(actions, e):
    n = len(actions)
    
    if random.random() < e:
        return random.randint(0, n-1)
    else:
        return np.argmax(actions)

In [14]:
n_actions = env.action_space.n
n_states = env.observation_space.n
Q = np.zeros([n_states, n_actions])

learning_rate = 0.85
y = 0.95
k = 10000.0

Q

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [5]:
r_total = 0
e_fun = interp1d([0, 40000], [1, 0.01], bounds_error=False, fill_value=0.01)
learning_rate_f = lambda t: k / (k + t)
global_step = 0

for t in range(200000):
    s = env.reset()
    
    while True:
        global_step += 1
        learning_rate = learning_rate_f(global_step)
        e = e_fun(global_step)
        
        a = next_action(Q[s], e)

        s1, r, done, info = env.step(a)
        r_total += r

        Q[s, a] += learning_rate * (r + y * np.max(Q[s1]) - Q[s, a])
        
        s = s1
        
        if done:
            break
            
    if t % 5000 == 0:
        print(r_total / 5000.0)
        r_total = 0

0.0
0.2524
0.6914


KeyboardInterrupt: 

In [ ]:
Q

In [11]:
s = env.reset()
done = False
for i in range(1000):
    a = next_action(Q[s], 0)
    s, r, done, info = env.step(a)
    env.render()
    print("")
    
    if done:
        break

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Right)

SFFF
FHFH
FFFH
HFFG
 

array([[ 0.19070067,  0.15914123,  0.16089736,  0.15916048],
       [ 0.06891698,  0.08796306,  0.08622338,  0.15390797],
       [ 0.14574003,  0.09735441,  0.115587  ,  0.11319179],
       [ 0.07115291,  0.07474071,  0.04844865,  0.10593075],
       [ 0.21871452,  0.14998555,  0.1644647 ,  0.12667895],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.11472175,  0.08650457,  0.17089026,  0.03366036],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.15019937,  0.21836031,  0.21510156,  0.27831993],
       [ 0.19862124,  0.39001742,  0.27015169,  0.21496852],
       [ 0.44423799,  0.31951613,  0.25206875,  0.16194338],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.30120626,  0.41727327,  0.50321368,  0.3678785 ],
       [ 0.52185656,  0.73616802,  0.60652192,  0.60124575],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])